In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import eradiate
from eradiate.experiments import CanopyAtmosphereExperiment
from eradiate.scenes.biosphere import load_rami_scenario
from eradiate.units import unit_registry as ureg

eradiate.set_mode("mono")

# Use the RAMI ID to reference the scenario
unpack_folder = "./rami_scenarios/"
scenario_data = load_rami_scenario(
    "HET14_WCO_UND", unpack_folder=unpack_folder, padding=1
)

srf = {
    "type": "delta",
    # "wavelengths": np.array([660.0, 550.0, 440.0]) * ureg.nm,
    "wavelengths": np.array([440.0, 550.0, 660.0]) * ureg.nm,
}

camera = {
    "type": "perspective",
    "film_resolution": [320, 240],
    "origin": [0, -10, 7],
    "target": [0, 0, 5],
    "up": [0, 0, 1],
    "srf": srf,
}

dhp = {
    "type": "hdistant",
    "film_resolution": [512, 512],
    "direction": [0, 0, -1],
    "target": [0, 0, 0.05],
    "ray_offset": 0.01,
    "srf": srf,
}

atmosphere = {
    "type": "heterogeneous",
    "molecular_atmosphere": {},
    "particle_layers": {"bottom": 0, "top": 5 * ureg.km, "tau_ref": 0.2},
}

directional = {"type": "directional", "zenith": 30.0, "azimuth": 45.0}
constant = {"type": "constant"}

scenario = CanopyAtmosphereExperiment(
    **scenario_data,
    atmosphere=None,
    measures=dhp,
    illumination=constant,
)

result = eradiate.run(scenario, spp=64)

In [ ]:
plt.imshow(
    eradiate.xarray.interp.dataarray_to_rgb(
        result["radiance"].squeeze(),
        channels=[("w", 660), ("w", 550), ("w", 440)],
        normalize=False,
    ) * 2
)
plt.axis("off")

In [ ]:
from eradiate.warp import uniform_hemisphere_to_square

def dhp_image(da, res: int = 512, x_label:str = "x", y_label:str = "y"):
    assert set(da.dims) == {"x", "y", "w"}
    result = np.zeros((res, res, 3))

    # Compute angles for each pixel
    pixel_centers = np.linspace(0, 1, res)
    film_x, film_y = np.meshgrid(pixel_centers, pixel_centers)
    film_center = np.array((0.5, 0.5))

    # distance to film center
    r = np.sqrt((film_x - film_center[0]) ** 2 + (film_y - film_center[1]) ** 2)

    # mask telling which pixels map to a valid (theta, phi) pair
    out = r > 0.5
    result[out] = np.nan

    # compute cosine zenith
    cos_theta = np.zeros((res, res))
    cos_theta[~out] = 1.0 - r[~out] * 2
    cos_theta[out] = np.nan

    # compute azimuth (in rad)
    phi = ((np.arctan2(film_y - film_center[0], film_x - film_center[1])) + np.pi)

    # derive direction vector
    d = eradiate.frame.cos_angle_to_direction(cos_theta.ravel(), phi.ravel())

    # Warp angles to film coordinates
    film_coords = uniform_hemisphere_to_square(d).reshape((res, res, 2))

    # Interpolate the data
    result = da.interp(
        **{
            x_label: (["x_index", "y_index"], film_coords[..., 0]),
            y_label: (["x_index", "y_index"], film_coords[..., 1]),
        },
    )

    # Drop non-dimension coordinates
    result = result.drop_vars(
        [name for name in da.coords if name not in result.dims]
    )

    return result


da = scenario.results["measure"]["radiance"].swap_dims(y_index="y", x_index="x").squeeze(drop=True)
da = dhp_image(da)
img = eradiate.xarray.interp.dataarray_to_rgb(
    da, channels=[("w", 660), ("w", 550), ("w", 440)], normalize=False
)
plt.imshow(img * 1.8)
plt.axis("off")
plt.show()

plt.imsave("dhp.png", np.clip(img * 1.8, 0, 1))